# Miscellaneous final figures panels related to the model

Plotted here:
- Figure 4, panel A: model curves for B16 tumors compared to E2aPBX
- Figure 6, panel B: model curves for PC-9 tumors and BEAS-2B with HHAT TCR antigens and Her2 CAR antigen
- Figure S3, panel G (top): SMARTA CAR T cells on B6 APCs compared to OT1/CD19 CAR on E2aPBX tumors
- Figure S5, panel H (left): model predictions for varying CD19 levels on Nalm6 tumors with NY-ESO/CD19 CAR T cells
- Figure S6, panel G (left): model antagonism for two TCR antigens (HHAT-WT, HHAT-L8F) and a CAR antigen (Her2)

# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.colors as clr
import seaborn as sns
import os, json, sys
import h5py
pj = os.path.join

if not ".." in sys.path:
    sys.path.insert(1, "..")
from utils.export import hdf5_to_dict

import warnings
warnings.filterwarnings('ignore')

In [ ]:
do_save_plots = False

root_dir = ".."
res_dir = pj(root_dir, "results", "for_plots")
fig_dir = "panels_misc"

In [ ]:
#@title Aesthetic parameters
sns.set_context('talk')
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False
plt.rcParams["figure.dpi"] = 80

# Figure 4A, left

In [ ]:
tcrcarFClabel = r"$FC_{\mathrm{TCR/CAR}}$"
tcrtcrFClabel = r"$FC_{\mathrm{TCR/TCR}}$"

In [ ]:
# Import data
df_model = pd.read_hdf(pj(res_dir, "model_antagonism_ratio_b16_e2apbx.h5"), key="df")
with open(pj(root_dir, "data", "pep_tau_map_ot1.json"), "r") as h:
    pep_tau_map = json.load(h)
highlight_taus = [0.001, pep_tau_map.get("V4"), pep_tau_map.get("N4")]
with open(pj(res_dir, "l_tcr_b16_e2apbx.json"), "r") as h:
    l_tcr_dict = json.load(h)

# Reformat index labels in df_model
df_model.index = df_model.index.rename({"TumorLine":"TumorLine", 
    "TCR_Antigen_Density":r"Simulated pulse concentration", "TCR_Antigen":r"$\tau_T$"})
df_model

In [ ]:
# Plot antagonism ratio vs tau for the L_TCR, L_CAR conditions of B16 tumors.
fig, ax = plt.subplots()
xvals = np.sort(df_model.index.get_level_values(r"TCR_Antigen_tau").unique().values)
ax.axhline(1.0, ls="--", color="k", lw=2.0)

tumor_colors = {"E2aPBX":"grey", "B16-CD19":"k"}
tumor_styles = {"E2aPBX":"--", "B16-CD19":"-"}
for tumor in list(df_model.index.get_level_values("TumorLine").unique())[::-1]:
    yvals = df_model.loc[(tumor, 1e6, xvals), "best"].values
    yvals_down = df_model.loc[(tumor, 1e6, xvals), "percentile_2.5"].values
    yvals_up = df_model.loc[(tumor, 1e6, xvals), "percentile_97.5"].values
    ax.plot(xvals, yvals, lw=4.0, color=tumor_colors[tumor], label=tumor,
           ls=tumor_styles[tumor])
    ax.fill_between(xvals, yvals_down, yvals_up, color=tumor_colors[tumor], alpha=0.2)

highlight_colors = ["g", "b", "r"]
highlight_labels = ["None", "V4", "N4"]
tumor_markers = {"B16-CD19":"o", "E2aPBX":"s"}
for i in range(len(highlight_taus)):
    #for tumor in df_model.index.get_level_values("TumorLine").unique():
    for tumor in ["B16-CD19", "E2aPBX"]:
        tau_tcr = highlight_taus[i]
        #lbl = highlight_labels[i] if tumor == "B16-CD19" else None
        ax.plot(tau_tcr, df_model.loc[(tumor, 1e6, tau_tcr), "best"],
                ls="none", marker=tumor_markers.get(tumor),
                mfc=highlight_colors[i], ms=10, mec=highlight_colors[i])

ax.set_xlabel(r"TCR antigenicity, $\tau$ (s)")
ax.set_ylabel(ylabel=tcrcarFClabel, size=22)
ax.set_yscale("log", base=2)
#ax.set_ylim([0.48, 2.05])
leg = ax.legend(frameon=False, loc="upper left", borderaxespad=0.0)#, fontsize=0.7*ax.yaxis.get_label().get_size())
for side in ["top", "right"]:
    ax.spines[side].set_visible(False)
ax.annotate('No\nTCR\nAg',xy=(-0.2,1.7),ha='left',va='center',color='g')
ax.annotate('V4',xy=(3.5,0.8),ha='left',va='center',color='b')
ax.annotate('N4',xy=(9,2.8),ha='left',va='center',color='r')
size_inches = fig.get_size_inches()
fig.set_size_inches(size_inches[0]*0.72, size_inches[1])
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(fig_dir, "4A-antagonism_b16_surface_molecule_numbers-narrow.pdf"), transparent=True,
          bbox_inches="tight", bbox_extra_artists=(leg,))
plt.show()
plt.close()

# Figure S3G

In [ ]:
#@title Load pre-computed model curves
df_smarta = pd.read_hdf(pj(res_dir, "model_curves_cd4.h5"), key="SMARTA_B6")
df_ot1 = pd.read_hdf(pj(res_dir, "model_curves_cd4.h5"), key="OT1_E2aPBX")
display(df_smarta)

In [ ]:
#@title Plot
# Plot model curves. TODO: adjust aesthetic details to match Sooraj's plots
fig, ax = plt.subplots(figsize=(5.5,5))
# Plot SMARTA vs B6

tau_tcr_range = df_smarta.index.get_level_values("TCR_Antigen_tau")

yvals = df_smarta.loc[(slice(None), tau_tcr_range), "best"].values
yvals_up = df_smarta.loc[(slice(None), tau_tcr_range), "percentile_97.5"].values
yvals_down = df_smarta.loc[(slice(None), tau_tcr_range), "percentile_2.5"].values

ax.plot(tau_tcr_range, yvals,
        color="k", label="SMARTA and B6", lw=3.0)
ax.fill_between(tau_tcr_range, yvals_down, yvals_up, color='k', alpha=0.3)

# Plot OT-1 vs E2aPBX
tau_tcr_range_ot1 = df_ot1.index.get_level_values("TCR_Antigen_tau")
yvals = df_ot1.loc[(slice(None), tau_tcr_range_ot1), "best"].values
yvals_up = df_ot1.loc[(slice(None), tau_tcr_range_ot1), "percentile_97.5"].values
yvals_down = df_ot1.loc[(slice(None), tau_tcr_range_ot1), "percentile_2.5"].values

ax.plot(tau_tcr_range_ot1, yvals,
        color="grey", label="OT-1 and E2aPBX", lw=3.0)
ax.fill_between(tau_tcr_range_ot1, yvals_down, yvals_up, color='grey', alpha=0.3)

ax.set(xlabel=r"TCR Antigen $\tau$ (s)", ylabel=r"$FC_{\mathrm{TCR/CAR}}$")
ax.axhline(1.0, ls="--", color="grey", lw=2.0)
ax.set_yscale("log", base=2)
ax.legend(title="CAR T cell and APC", frameon=False)
if do_save_plots:
    fig.savefig(pj(fig_dir, 'suppFigure3G-top_modelParameterization.pdf'),
                bbox_inches='tight', transparent=True)
plt.show()
plt.close()

# Figure S6G

In [ ]:
# Load pre-computed model predictions
df_model_mix = pd.read_hdf(pj(res_dir, "tcr_antigen_mixture_car_antagonism_model.h5"), key="range")
df_model_mix

In [ ]:
# Plot results: heatmap of antagonism/enhancement. TODO: change color map and other aesthetics
log2_fcs = np.log2(df_model_mix.droplevel(["Ag_1", "Ag_2"]))
log2_fcs = log2_fcs.unstack("Ag_2_density_M")  # Agonist is in columns
max_ampli = np.abs(log2_fcs.values).max()
normalizer = mpl.colors.Normalize(vmin=-max_ampli, vmax=max_ampli)

fig, ax = plt.subplots(figsize=(7.5,7.5))
ax.spines["top"].set_visible(True)
ax.spines["right"].set_visible(True)
im = ax.imshow(log2_fcs, norm=normalizer, cmap="PuOr_r", origin="lower", interpolation="gaussian",
              extent=(-12, -4, -12, -4),zorder=0)
ax.set(xlabel=r"HHAT-p8F pulse concentration (M)", ylabel=r"HHAT-WT pulse concentration (M)")
ax.set_xticks(np.arange(-12, -3, 1))
ax.set_xticklabels([r"$10^{" + str(a) + r"}$" for a in ax.get_xticks()])
ax.set_yticks(np.arange(-12, -3, 1))
ax.set_yticklabels([r"$10^{" + str(a) + r"}$" for a in ax.get_yticks()])

cbar = fig.colorbar(im, label=r"Model-predicted $FC_{\mathrm{TCR/CAR}}$",fraction=0.046, pad=0.04)
cbar_ticks = np.arange(-int(np.floor(max_ampli)), int(np.floor(max_ampli))+1, 1)
cbar.ax.set_yticks(cbar_ticks)
cbar.ax.set_yticklabels(["$2^{" + str(int(a)) + r"}$" for a in cbar_ticks])

plt.plot(np.log10(5*1e-6),np.log10(5*1e-6),marker='o',color='purple',ms=25,zorder=100)
plt.plot(-5,np.log10(1*1e-11),marker='o',color='red',ms=25,zorder=100)
plt.plot(np.log10(1*1e-11),-5,marker='o',color='blue',ms=25,zorder=100)

if do_save_plots:
    fig.savefig(pj(fig_dir, "S6G_tcr-car_antagonism_model_hhat_mix_heatmap.pdf"),
          transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Figure 6B, right

In [ ]:
# Specific aesthetics
sns.set_context('poster')
sns.set_style('ticks')

In [ ]:
#@title Load data to plot
df_model_aebs = pd.read_hdf(pj(res_dir, "model_predict_aebs_hhat_wt_neoantigen.h5"), key="df")
df_model_aebs.index = df_model_aebs.index.set_names(["Target", "Pulse concentration (uM)", r"$\tau^T$"])

with open(pj(res_dir, "hhat_peptides_params_aebs_wt_neoantigen.json"), "r") as h:
    hhat_params_dict = json.load(h)

In [ ]:
#@title Plot
# Data on L_T, L_C for both APC lines
size_factor_her = 1.0  # Unsure of the size, don't correct because qualitative anyways
mtc = "Geometric mean"
hla_levels = hhat_params_dict["hla_levels"]
her2_levels = hhat_params_dict["her2_levels"]
hhat_pep_taus = hhat_params_dict["hhat_pep_taus"]
tumor_peptide_map = hhat_params_dict["tumor_peptide_map"]
her2_tau = hhat_params_dict["her2_tau"]

# Plot results: just two antagonism ratio curves vs tau, highlight the peptides
# Aesthetic parameters
tumor_palette = {
    "BEAS2B-GL": "#CED5C3",  # pale green from fig. 5
    #"BEAS2B-GL": (167/255, 177/255, 112/255), # Slightly darker green (nucleus in fig. 5)
    #"PC9-GL": "#DFBCA8",  # pale pink from fig. 5
    "PC9-GL": (195/255, 156/255, 142/255),   # Slightly darker pink (nucleus in fig. 5)
}
peptide_palette = {
    "HHAT-WT": "b",
    "HHAT-p8F": "r"
}
markerstyles = {
    "BEAS2B-GL":"X",
    "PC9-GL": "o"
}
linestyles = {
    "BEAS2B-GL": "--",
    "PC9-GL": "-"
}

fig, ax = plt.subplots(figsize=(4.5,5))

peptide_labels = {
    "PC9-GL": r"HHAT${}^\mathregular{L8F}$" + "\n(Strong)",# (Neoantigen)",
    "BEAS2B-GL": r"HHAT${}^\mathregular{WT}$" + " (Weak)"  # (Self-antigen)",
}
# Nicer name for tumors
tumor_labels = {
    "BEAS2B-GL": "BEAS-2B (Tissue)",
    "PC9-GL": "PC9 (Tumor)"
}

ax.axhline(1.0, ls=":", lw=1.0, color="k")
annotList = []
for tumor in tumor_palette:
    xvals = np.sort(df_model_aebs.xs(tumor, level="Target").index
            .get_level_values(r"$\tau^T$").unique().values)
    yvals = df_model_aebs.loc[(tumor, slice(None), xvals), "best"].values
    yvals_up = df_model_aebs.loc[(tumor, slice(None), xvals), "percentile_97.5"].values
    yvals_down = df_model_aebs.loc[(tumor, slice(None), xvals), "percentile_2.5"].values
    ax.plot(xvals, yvals, label=tumor_labels[tumor], color=tumor_palette[tumor],
           ls=linestyles[tumor], alpha=1, lw=4.0)
    ax.fill_between(xvals, yvals_down, yvals_up, color=tumor_palette[tumor], alpha=0.3)
    # highlight
    tau_mark = hhat_pep_taus[tumor_peptide_map[tumor]]
    mark_color = peptide_palette[tumor_peptide_map[tumor]]
    ymark = df_model_aebs.loc[(tumor, slice(None), tau_mark), "best"].values
    yfact = 0.7 if tumor == "PC9-GL" else 1.0
    ax.plot([tau_mark], ymark, ms=16,
           ls="none", marker=markerstyles[tumor], mfc=mark_color, mec=mark_color)
    a = ax.annotate(peptide_labels[tumor], xy=(tau_mark+0.75, ymark*yfact),
                ha="left", va="center", color=mark_color)
    annotList.append(a)
for side in ["top", "right"]:
    ax.spines[side].set_visible(False)
ax.set_xlabel(r"TCR antigenicity, $\tau$ (s)")
ax.set_ylabel(tcrcarFClabel)
ax.set_yscale("log", base=2)
#ax.set_ylim([0.48, 2.05])
leg = ax.legend(frameon=False, loc="center", bbox_to_anchor=(0.5, 1.1))
ax.axhline(1.0, ls="--", color="k")
antagonism_threshold = 0.5
enhancement_threshold = 2.0
ylims = ax.get_ylim()
ax.set_ylim(ylims)

ax.axhline(color='k',linestyle=':',y=antagonism_threshold)
ax.axhline(color='k',linestyle=':',y=enhancement_threshold)
ogXlim = ax.get_xlim()
ax.fill_between([ax.get_xlim()[0]-5,ax.get_xlim()[1]+5], ylims[0], antagonism_threshold, color="purple", alpha=0.2)
ax.fill_between([ax.get_xlim()[0]-5,ax.get_xlim()[1]+5], enhancement_threshold, ylims[1], color="orange", alpha=0.2)
ax.set_xlim(ogXlim)

figsize = fig.get_size_inches()
fig.set_size_inches(figsize[0]*1.1, figsize[1]*0.95)
#fig.tight_layout()

if do_save_plots:
    fig.savefig(pj(fig_dir, "6B_right-antagonism_hhat_peptides_beas2b_pc9_numbers-condensed.pdf"), transparent=True,
           bbox_inches="tight", bbox_extra_artists=(leg,annotList[0],annotList[1]))
plt.show()
plt.close()

# Figure S5H, left

In [ ]:
tumor_order = ["CD19_High", "CD19_Med", "CD19_Low", "CD19_KO"]
tumor_colors = sns.color_palette("viridis_r", n_colors=len(tumor_order))
tumor_colors = {t:tumor_colors[i] for i, t in enumerate(tumor_order)}

In [ ]:
#@title Load data
df_model = pd.read_hdf(pj(res_dir, "model_antagonism_varying_cd19_human_cells.h5"), key="df")
display(df_model)
df_model.index = df_model.index.rename({"TCR_Antigen_Density_uM":"TCR Antigen Density", "TCR_Antigen": r"TCR Antigen $\tau$"})

with h5py.File(pj(res_dir, "model_antagonism_varying_cd19_human_cells.h5"), "r") as h:
    grp = h.get("cd19_levels")
    cd19_levels_nalm6 = hdf5_to_dict(grp)
    grp = h.get("mhc_levels")
    mhc_levels_nalm6 = grp.get("mhc_level")[()]

df_model

In [ ]:
#@title Plot
# Plot antagonism ratio vs tau for the L_TCR, L_CAR conditions of B16 tumors.
fig, ax = plt.subplots()
size_inches = fig.get_size_inches()
fig.set_size_inches(0.8*size_inches[0], size_inches[1])
xvals = np.sort(df_model.index.get_level_values(r"TCR_Antigen_tau").unique().values)

# Default seaborn dash styles
# Using dashes keyword:
# https://matplotlib.org/stable/gallery/lines_bars_and_markers/line_demo_dash_control.html
tumor_styles = [
    "",
    (4, 1.5),
    (1, 1),
    (3, 1.25, 1.5, 1.25)
    #(5, 1, 1, 1)
  ]
# Default seaborn marker styles
marker_styles = ["o", "X", (4, 0, 45), "P"]
tumor_styles = {t:tumor_styles[i] for i, t in enumerate(tumor_order)}
marker_styles = {t:marker_styles[i] for i, t in enumerate(tumor_order)}

nice_tumor_lbls = {
    "CD19_High":r"CD19${}^{\mathrm{Hi}}$",
    "CD19_Med":r"CD19${}^{\mathrm{Int}}$",
    "CD19_Low":r"CD19${}^{\mathrm{Lo}}$",
    "CD19_KO":r"CD19${}^{\mathrm{KO}}$",
}

plot_conc = 1e0  # 1uM
for tumor in df_model.index.get_level_values("Tumor").unique():
    yvals = df_model.loc[(tumor, plot_conc, xvals), "best"].values
    yvals_up = df_model.loc[(tumor, plot_conc, xvals), "percentile_97.5"].values
    yvals_down = df_model.loc[(tumor, plot_conc, xvals), "percentile_2.5"].values
    ax.plot(xvals, yvals, color=tumor_colors[tumor], label=nice_tumor_lbls[tumor],
           dashes=tumor_styles[tumor], lw=4.0)
    ax.fill_between(xvals, yvals_down, yvals_up, color=tumor_colors[tumor], alpha=0.2)

    # Annotate the position of NY-ESO peptides
    # Mark the last point as a test for now
    #ax.plot(xvals[-1], yvals[-1], color=tumor_colors[tumor],
    #        marker=marker_styles[tumor], mec="w", mew=1.0)
for side in ["top", "right"]:
    ax.spines[side].set_visible(False)
ylbl = ax.set_ylabel(r"FC${}_{TCR / CAR}$")
ax.set_xlabel(r"TCR antigenicity, $\tau$ (s)", fontsize=ylbl.get_fontsize()*0.9)
ax.set_yscale("log", base=2)
#ax.set_ylim([0.48, 2.05])
#ax.legend(frameon=False, loc="upper left", bbox_to_anchor=(1.0, 1.0))
ax.axhline(1.0, ls="--", color="k")
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(fig_dir, "S5H_left-NYESOmodelOutput.pdf"), transparent=True,
            bbox_inches="tight", bbox_extra_artists=(ax.xaxis.label,))
plt.show()
plt.close()